# Spark Session

Spark can work with data located on HDFS or a non-distributed filesystem. It can also use YARN from Hadoop, or [Mesos](https://mesos.apache.org/), or a resource manager of its own.

All distributed operations with Spark are done using so-called Spark Session. Usually one is already created by your cluster's administrator:

In [11]:
from pyspark.sql import SparkSession

spark1 = SparkSession.builder.getOrCreate()

In [12]:
spark1

In [13]:
spark1.stop()

In [11]:
!hdfs dfs -ls /

Found 3 items
drwx------   - mapred hadoop          0 2020-11-26 12:44 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2020-11-26 12:44 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2020-11-26 13:09 /user


In [1]:
spark

# Reading Data

Spark can consume data in a variety of formats, e.g. in JSON. We use the [YELP Dataset](https://www.yelp.com/dataset) for this example. It's easily obtainable and free to use in education and research.

In [2]:
reviews_on_hdfs = "/user/borisshminke/data/yelp_academic_dataset_review.json"

In [3]:
%%time

spark.read.text(reviews_on_hdfs).count()

CPU times: user 2.03 ms, sys: 6.51 ms, total: 8.54 ms
Wall time: 11.3 s


8021122

In [4]:
some = spark.read.text(reviews_on_hdfs)
some.show(n=2)

+--------------------+
|               value|
+--------------------+
|{"review_id":"xQY...|
|{"review_id":"UmF...|
+--------------------+
only showing top 2 rows



This code simply reads a JSON file as a text, line by line, and counts the number of lines. Let's compare the speed with `wc` tool:

In [4]:
%%time

!wc -l /home/borisshminke/Downloads/yelp_academic_dataset_review.json

8021122 /home/borisshminke/Downloads/yelp_academic_dataset_review.json
CPU times: user 70.3 ms, sys: 8.76 ms, total: 79.1 ms
Wall time: 2.12 s


Although `wc` is implemented in C and is more efficient in general than JVM code behind Spark, it uses only one CPU, and sometimes may work slower than it's distributed counterpart from Spark.

Parsing JSON in Spark is really simple:

In [6]:
reviews = spark.read.json(reviews_on_hdfs)
reviews.show(n=5)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|-MhfebM0QIsKt87iD...|   0|2015-04-15 05:21:16|    0|xQY8N_XvtGbearJ5X...|  2.0|As someone who ha...|     5|OwjRMXRC0KyPrIlcj...|
|lbrU8StCq3yDfr-QM...|   0|2013-12-07 03:16:52|    1|UmFMZ8PyXZTY2Qcwz...|  1.0|I am actually hor...|     1|nIJD_7ZXHq-FX8byP...|
|HQl28KMwrEKHqhFrr...|   0|2015-12-05 03:18:11|    0|LG2ZaYiOgpr2DK_90...|  5.0|I love Deagan's. ...|     1|V34qejxNsCbcgD8C0...|
|5JxlZaqCnk1MnbgRi...|   0|2011-05-27 05:30:52|    0|i6g_oA9Yf9Y31qt0w...|  1.0|Dismal, lukewarm,...|     0|ofKDkJKXSKZXu5xJN...|
|IS4cv902ykd8wj1TR...|   0|2017-01-14 21:56:57|    0|6TdNDKywdbjoTkize...|  4.0|Oh happy d

In [9]:
from pyspark.sql import functions as sf

x = reviews \
    .groupby(sf.date_trunc("day", "date").alias("day")) \
    .agg(sf.sum("cool").alias("total_cool")) \
    .sort(sf.desc("total_cool"))

In [40]:
# Spark can be used similarly to Pandas

from pyspark.sql import functions as sf

(
    reviews
    .select(sf.col("cool").alias("groovy"), "date")
    .groupby(sf.date_trunc("day", "date").alias("day"))
    .agg(
        sf.count(sf.col("groovy")).alias("total_cool")
    )
    .sort(sf.desc("total_cool"))
    .show(n=5)
)

+-------------------+----------+
|                day|total_cool|
+-------------------+----------+
|2019-02-17 00:00:00|      4915|
|2018-08-05 00:00:00|      4737|
|2017-08-12 00:00:00|      4698|
|2018-03-11 00:00:00|      4696|
|2018-07-07 00:00:00|      4661|
+-------------------+----------+
only showing top 5 rows



In [7]:
# or you can use Spark as an SQL engine

reviews.createOrReplaceTempView("reviews")
spark.sql("""
SELECT
    date_trunc('day', date) AS day,
    SUM(cool) AS total_cool
FROM reviews
GROUP BY
    day
ORDER BY
    total_cool DESC
LIMIT 5
""").show()

+-------------------+----------+
|                day|total_cool|
+-------------------+----------+
|2019-01-06 00:00:00|      3133|
|2019-01-02 00:00:00|      3017|
|2018-04-09 00:00:00|      2897|
|2018-03-13 00:00:00|      2810|
|2018-04-23 00:00:00|      2709|
+-------------------+----------+



# Do It Yourself

[Spark Manual](http://spark.apache.org/docs/2.4.3/api/python/index.html) is your best friend!

* count number of users, buisenesses
* count average number of reviews and stars per business and per user
* find histograms for distributions of cool, funny, and useful columns
* find ten most frequent words from the reviews
* save results to disk

In [12]:
reviews.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [20]:
new_data = reviews.select("user_id")

# Action

count, show, top, max, min - computation happens

# Transformations

whatever - computation doesn't happen

In [14]:
reviews.select(sf.col("user_id"))

DataFrame[user_id: string]

In [21]:
reviews.count()

8021122

In [23]:
reviews = reviews.cache()

In [25]:
%%time

reviews.count()

CPU times: user 7.5 ms, sys: 1.43 ms, total: 8.92 ms
Wall time: 27.3 s


8021122

In [26]:
%%time

reviews.count()

CPU times: user 1.47 ms, sys: 279 µs, total: 1.75 ms
Wall time: 257 ms


8021122

In [31]:
users = reviews.select("user_id").distinct().cache()

In [32]:
%%time

users.count()

CPU times: user 0 ns, sys: 3.75 ms, total: 3.75 ms
Wall time: 7.31 s


1968703

In [33]:
%%time

users.count()

CPU times: user 2.56 ms, sys: 0 ns, total: 2.56 ms
Wall time: 443 ms


1968703

In [34]:
users.explain()

== Physical Plan ==
InMemoryTableScan [user_id#30]
   +- InMemoryRelation [user_id#30], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(2) HashAggregate(keys=[user_id#30], functions=[])
            +- Exchange hashpartitioning(user_id#30, 200)
               +- *(1) HashAggregate(keys=[user_id#30], functions=[])
                  +- InMemoryTableScan [user_id#30]
                        +- InMemoryRelation [business_id#22, cool#23L, date#24, funny#25L, review_id#26, stars#27, text#28, useful#29L, user_id#30], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
                              +- *(1) FileScan json [business_id#22,cool#23L,date#24,funny#25L,review_id#26,stars#27,text#28,useful#29L,user_id#30] Batched: false, Format: JSON, Location: InMemoryFileIndex[hdfs://cluster-444c-m/user/borisshminke/data/yelp_academic_dataset_review.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<business_id:string,cool:bigint,date:stri

In [36]:
(
    reviews
    .agg(sf.approx_count_distinct(reviews.user_id).alias('distinct_users'))
    .collect()
)

[Row(distinct_users=1910451)]

In [53]:
reviews.select(
    sf.split(sf.col("text"), " ").alias("words")
).show(n=2)

+--------------------+
|               words|
+--------------------+
|[As, someone, who...|
|[I, am, actually,...|
+--------------------+
only showing top 2 rows



In [54]:
(
    reviews.select(
        sf.split(sf.col("text"), " ").alias("words")
    )
    .select(
        sf.explode("words").alias("word")
    )
).show(n=5)

+-------+
|   word|
+-------+
|     As|
|someone|
|    who|
|    has|
| worked|
+-------+
only showing top 5 rows



In [52]:
(
    reviews.select(
        sf.split(sf.col("text"), " ").alias("words")
    )
    .filter("stars > 4")
    .select(
        sf.explode("words").alias("word")
    )
    .cache()
    .filter("word NOT IN ('the', 'and', 'to', 'I', 'a', 'was', 'of', 'is', 'for')")
    .groupby(
        "word"
    )
    .agg(sf.count("word").alias("cnt"))
    .sort(sf.desc("cnt"))
    .show(n=10)
)

+----+--------+
|word|     cnt|
+----+--------+
|    |13151079|
|  in| 9041099|
|  it| 7460125|
|  my| 7109453|
|with| 6964448|
|that| 6752534|
| The| 6302953|
| but| 5685251|
|  on| 5678107|
|have| 5335047|
+----+--------+
only showing top 10 rows



In [66]:
spark.sql("""
SELECT
    explode(split(text, ' '´)) AS word,
    COUNT(*) cnt
FROM reviews
GROUP BY
    word
ORDER BY
    cnt DESC
LIMIT 5
""").show()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ParseException: u"\nextraneous input '\xb4' expecting {')', ','}(line 3, pos 27)\n\n== SQL ==\n\nSELECT\n    explode(split(text, ' '\xb4)) AS word,\n---------------------------^^^\n    COUNT(*) cnt\nFROM reviews\nGROUP BY\n    word\nORDER BY\n    cnt DESC\nLIMIT 5\n"

In [56]:
users.write.csv("/user/borisshminke/data/users")

In [57]:
!hdfs dfs -ls /user/borisshminke/data/users

Found 201 items
-rw-r--r--   2 root hadoop          0 2020-11-26 15:12 /user/borisshminke/data/users/_SUCCESS
-rw-r--r--   2 root hadoop     229540 2020-11-26 15:12 /user/borisshminke/data/users/part-00000-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     227516 2020-11-26 15:12 /user/borisshminke/data/users/part-00001-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     224894 2020-11-26 15:12 /user/borisshminke/data/users/part-00002-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     226021 2020-11-26 15:12 /user/borisshminke/data/users/part-00003-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     222042 2020-11-26 15:12 /user/borisshminke/data/users/part-00004-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     225331 2020-11-26 15:12 /user/borisshminke/data/users/part-00005-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     225492 

-rw-r--r--   2 root hadoop     229724 2020-11-26 15:12 /user/borisshminke/data/users/part-00159-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     225216 2020-11-26 15:12 /user/borisshminke/data/users/part-00160-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     227010 2020-11-26 15:12 /user/borisshminke/data/users/part-00161-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     226389 2020-11-26 15:12 /user/borisshminke/data/users/part-00162-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     228965 2020-11-26 15:12 /user/borisshminke/data/users/part-00163-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     228942 2020-11-26 15:12 /user/borisshminke/data/users/part-00164-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r--r--   2 root hadoop     231541 2020-11-26 15:12 /user/borisshminke/data/users/part-00165-df5777f8-9c80-400d-b56b-552b22216437-c000.csv
-rw-r-

In [61]:
new = spark.read.csv("/user/borisshminke/data/users")

In [62]:
new.show(n=2)

+--------------------+
|                 _c0|
+--------------------+
|iAevEfcmtCr2cpd6I...|
|h-Zj9GUJvErqVOw-K...|
+--------------------+
only showing top 2 rows



In [59]:
!hdfs dfs -getmerge /user/borisshminke/data/users /home/borisshminke/Downloads/users

In [60]:
!head /home/borisshminke/Downloads/users

KWFiZKiZBANVxuhm4MLBNw
VmYpF5C3GL-7wFnvOkhpvg
1Dul59QEe-Q-7OQHTLOptw
xS6kmkMXp0PRrFwkSWq2-w
j56G3m8vYtA_2Io6FcISBg
ruHz-qN-j21kg0iyIgGE9Q
M6-A6F0B3kM5i94Kr0XHcw
z2Gi5vo-8j544qN_g6ziEg
CzkWUMIYDxUSetfCRvYG5g
Uf_TVv1Z4s024jdI4UKWtA
